# Entrenament de Models - Classificació de Pingüins Palmer

Aquest notebook entrena 4 models diferents per classificar les espècies de pingüins:
1. Regressió Logística
2. SVM (Support Vector Machine)
3. Arbres de Decisió (Decision Trees)
4. KNN (k-Nearest Neighbours)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pickle

# Carregar el dataset net
df = pd.read_csv('../datasets/penguins_clean.csv')
print(f"Dataset carregat amb {len(df)} files")
df.head()

## 1. Preparació de les dades

In [ ]:
# Definir variables categòriques i numèriques
categorical = ['island', 'sex']
numerical = ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']

print("Variables categòriques:", categorical)
print("Variables numèriques:", numerical)

In [ ]:
# Codificar la variable objectiu (species)
le = LabelEncoder()
df['species_encoded'] = le.fit_transform(df['species'])

print("\nCodificació d'espècies:")
for i, species in enumerate(le.classes_):
    print(f"{i}: {species}")

In [ ]:
# Dividir en entrenament (80%) i prova (20%)
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42, stratify=df['species_encoded'])

print(f"\nDades d'entrenament: {len(df_train)} ({len(df_train)/len(df)*100:.1f}%)")
print(f"Dades de prova: {len(df_test)} ({len(df_test)/len(df)*100:.1f}%)")

# Separar X i y
y_train = df_train['species_encoded'].values
y_test = df_test['species_encoded'].values

print(f"\nDistribució d'espècies en entrenament:")
print(pd.Series(y_train).value_counts().sort_index())
print(f"\nDistribució d'espècies en prova:")
print(pd.Series(y_test).value_counts().sort_index())

## 2. Enginyeria de propietats (Feature Engineering)

In [ ]:
# Convertir a diccionaris per DictVectorizer
train_dict = df_train[categorical + numerical].to_dict(orient='records')
test_dict = df_test[categorical + numerical].to_dict(orient='records')

# Codificació one-hot amb DictVectorizer
dv = DictVectorizer(sparse=False)
dv.fit(train_dict)

X_train = dv.transform(train_dict)
X_test = dv.transform(test_dict)

print(f"\nForma de X_train: {X_train.shape}")
print(f"Forma de X_test: {X_test.shape}")
print(f"\nCaracterístiques generades:")
print(dv.get_feature_names_out())

In [ ]:
# Normalitzar les variables numèriques
scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"\nEstadístiques de X_train_scaled:")
print(f"Mitjana: {X_train_scaled.mean(axis=0).mean():.4f}")
print(f"Desviació estàndard: {X_train_scaled.std(axis=0).mean():.4f}")

## 3. Entrenament dels Models

In [ ]:
# Funció per avaluar models
def evaluate_model(model, X_test, y_test, model_name):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    print(f"\n{'='*60}")
    print(f"MODEL: {model_name}")
    print(f"{'='*60}")
    print(f"\nAccuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
    print(f"\nInforme de classificació:")
    print(classification_report(y_test, y_pred, target_names=le.classes_))
    print(f"\nMatriu de confusió:")
    print(confusion_matrix(y_test, y_pred))
    
    return accuracy

### 3.1. Regressió Logística

In [ ]:
# Entrenar Regressió Logística
lr_model = LogisticRegression(max_iter=1000, random_state=42)
lr_model.fit(X_train_scaled, y_train)

# Avaluar
lr_accuracy = evaluate_model(lr_model, X_test_scaled, y_test, "Regressió Logística")

### 3.2. SVM (Support Vector Machine)

In [ ]:
# Entrenar SVM
svm_model = SVC(kernel='rbf', random_state=42, probability=True)
svm_model.fit(X_train_scaled, y_train)

# Avaluar
svm_accuracy = evaluate_model(svm_model, X_test_scaled, y_test, "SVM")

### 3.3. Arbres de Decisió (Decision Trees)

In [ ]:
# Entrenar Arbres de Decisió
dt_model = DecisionTreeClassifier(random_state=42, max_depth=5)
dt_model.fit(X_train_scaled, y_train)

# Avaluar
dt_accuracy = evaluate_model(dt_model, X_test_scaled, y_test, "Arbres de Decisió")

### 3.4. KNN (k-Nearest Neighbours)

In [ ]:
# Entrenar KNN
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train_scaled, y_train)

# Avaluar
knn_accuracy = evaluate_model(knn_model, X_test_scaled, y_test, "KNN")

## 4. Comparació de Models

In [ ]:
# Comparar resultats
results = pd.DataFrame({
    'Model': ['Regressió Logística', 'SVM', 'Arbres de Decisió', 'KNN'],
    'Accuracy': [lr_accuracy, svm_accuracy, dt_accuracy, knn_accuracy]
})

results = results.sort_values('Accuracy', ascending=False).reset_index(drop=True)
print("\n" + "="*60)
print("RESUM DE RESULTATS")
print("="*60)
print(results.to_string(index=False))
print(f"\nMillor model: {results.iloc[0]['Model']} amb {results.iloc[0]['Accuracy']*100:.2f}% d'accuracy")

## 5. Serialització dels Models

In [ ]:
# Guardar tots els models
models = {
    'logistic_regression': (dv, scaler, le, lr_model),
    'svm': (dv, scaler, le, svm_model),
    'decision_tree': (dv, scaler, le, dt_model),
    'knn': (dv, scaler, le, knn_model)
}

for model_name, model_tuple in models.items():
    filename = f'../models/{model_name}_model.pkl'
    with open(filename, 'wb') as f:
        pickle.dump(model_tuple, f)
    print(f"Model guardat: {filename}")

print("\n✓ Tots els models han estat serialitzats correctament!")